In [1]:
import re
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz, process

%matplotlib inline

In [170]:
h1b = pd.read_csv("../dat/clean_naics_employer.csv")

In [171]:
h1b = h1b.drop(columns=["Unnamed: 0", "status", "full_time_pos"])

In [117]:
h1b.groupby("lca_case_soc_code").size().sort_values(ascending=False).iloc[150:200]

lca_case_soc_code
13-1023       144
25-1065       144
25-1199       142
11-9051       141
17-2151       139
21-1091       139
15-1035       138
17-3023       138
19-2021       136
21-1012       136
27-1029       135
27-3099       131
41-4011       130
25-1122       129
17-1012       127
25-2054       127
15-1121.00    127
25-1126       127
19-3051       125
17-3029       124
27-3043       122
29-1064       118
1981-11-01    115
15-1051.00    114
25-4012       112
19-1041       112
21-1014       112
29-1029       110
25-1066       108
25-1125       105
11-3011       104
2013-11-01    103
19-4041       101
19-1011        92
13-1141        91
19-4099        91
35-1011        88
21-1015        88
23-2099        87
19-3031        86
13-1111.00     86
25-2053        85
1932-11-01     85
19-4031        83
19-3022        82
29-1031        82
25-1062        80
13-2011.01     79
13-1022        78
21-1021        78
dtype: int64

In [61]:
# some of these are very clearly wrong....
h1b.loc[h1b.lca_case_soc_code == '1941-11-01', ["lca_case_soc_name"]].groupby("lca_case_soc_name").size().sort_values(ascending=False)

lca_case_soc_name
Architectural and Engineering Managers                1230
Property, Real Estate, and Community Association M      62
dtype: int64

In [120]:
h1b.loc[h1b.lca_case_soc_code == '2013-11-01', ["lca_case_soc_name"]].groupby("lca_case_soc_name").size().sort_values(ascending=False)

lca_case_soc_name
Farmers, Ranchers, and Other Agricultural Managers    103
dtype: int64

In [20]:
# grab the correct ones
res = [re.findall(r"^\d{2}-\d{4}", soc) for soc in h1b["lca_case_soc_code"].fillna("---").values]

In [43]:
# see what's left after
badsoc = h1b["lca_case_soc_code"].fillna("---").values.flatten()[np.where(np.array([len(r) for r in res]) == 0)[0]]

In [45]:
set(badsoc)

{'11-304',
 '111021',
 '112021',
 '113021',
 '113031',
 '113061',
 '1131',
 '131041',
 '131081',
 '131161',
 '132011',
 '132051',
 '15--1132',
 '15-112',
 '15-119.01',
 '15-119.08',
 '15.1131',
 '15.1199.01',
 '15.1199.02',
 '15.1199.09',
 '15/1121',
 '15/1199.01',
 '151031',
 '1511-31',
 '1511.01',
 '151121',
 '151131',
 '151132',
 '151133',
 '151141',
 '151142',
 '151150',
 '151199',
 '152031',
 '152041',
 '1702141',
 '171012',
 '172071',
 '172072',
 '172141',
 '19-104',
 '191021',
 '191042',
 '192031',
 '192032',
 '1931-11-01',
 '1932-11-01',
 '1933-11-01',
 '1939-11-01',
 '1941-11-01',
 '1951-11-01',
 '1961-11-01',
 '1971-11-01',
 '1981-11-01',
 '1999-11-01',
 '2',
 '2011-11-01',
 '2013-11-01',
 '2021-11-01',
 '2022-11-01',
 '251022',
 '252021',
 '27.1014',
 '27.3042.00',
 '271011',
 '273091',
 '29.1062',
 '291051',
 '291062',
 '291063',
 '291066',
 '291069',
 '291122',
 '291123',
 '291127',
 '291131',
 '292011',
 '351011',
 '41-012'}

In [51]:
# It's a small number, so I'll mostly scrap them but some are fairly common, so let's figure those out
h1b.loc[h1b.lca_case_soc_name == 'Financial Managers', ["lca_case_soc_code"]].groupby("lca_case_soc_code").size().sort_values(ascending=False)

lca_case_soc_code
1931-11-01    1286
11-3031.00      15
dtype: int64

In [172]:
# the correct code for financial managers, after checkign on ONET, is 11-3031, so dunno where this value came out of
h1b.loc[h1b.lca_case_soc_code == '1931-11-01', ["lca_case_soc_code"]] = '11-3031'

In [57]:
#h1b.loc[h1b.lca_case_soc_name == 'Computer and Information Systems Managers', ["lca_case_soc_code"]].groupby("lca_case_soc_code").size().sort_values(ascending=False)
h1b.loc[h1b.lca_case_soc_name == 'Marketing Managers', ["lca_case_soc_code"]].groupby("lca_case_soc_code").size().sort_values(ascending=False)

lca_case_soc_code
2021-11-01    2021
11-2021.00      25
dtype: int64

In [173]:
# In this case, I'm gonna guess that it could have been either 11-3021 for Computer Managers
# or 11-2021 for Marrketing Managers, so I'll fix accordingly
h1b.loc[h1b.lca_case_soc_name == 'Computer and Information Systems Managers', ["lca_case_soc_code"]] = '11-3021'
h1b.loc[h1b.lca_case_soc_name == 'Marketing Managers', ["lca_case_soc_code"]] = '11-2021'

In [174]:
# grabbing the correct value from ONET again
h1b.loc[h1b.lca_case_soc_name == 'Architectural and Engineering Managers', ["lca_case_soc_code"]] = '11-9041'

In [175]:
# And again - it's hard to tell whether people are better at putting in the name or the code...
# probably the names are more reliable
h1b.loc[h1b.lca_case_soc_name == 'Property, Real Estate, and Community Association M', ["lca_case_soc_code"]] = '11-9141'

In [176]:
# fix a couple more of the big ones first - all freakin managerial positions for some reason...
h1b.loc[h1b.lca_case_soc_name == 'Medical and Health Services Managers', ["lca_case_soc_code"]] = '11-9111'
h1b.loc[h1b.lca_case_soc_name == 'Advertising and Promotions Managers', ["lca_case_soc_code"]] = '11-2011'
h1b.loc[h1b.lca_case_soc_name == 'Administrative Services Managers', ["lca_case_soc_code"]] = '11-3011'
h1b.loc[h1b.lca_case_soc_name == 'Compensation and Benefits Managers', ["lca_case_soc_code"]] = '11-3111'
h1b.loc[h1b.lca_case_soc_name == 'Managers, All Other', ["lca_case_soc_code"]] = '11-9199'
h1b.loc[h1b.lca_case_soc_name == 'Sales Managers', ["lca_case_soc_code"]] = '11-2022'
h1b.loc[h1b.lca_case_soc_name == 'Industrial Production Managers', ["lca_case_soc_code"]] = '11-3051'
h1b.loc[h1b.lca_case_soc_name == 'Social and Community Service Managers', ["lca_case_soc_code"]] = '11-9151'
h1b.loc[h1b.lca_case_soc_name == 'Food Service Managers', ["lca_case_soc_code"]] = '11-9051'
h1b.loc[h1b.lca_case_soc_name == 'Construction Managers', ["lca_case_soc_code"]] = '11-9021'
h1b.loc[h1b.lca_case_soc_name == 'Natural Sciences Managers', ["lca_case_soc_code"]] = '11-9121'
h1b.loc[h1b.lca_case_soc_name == 'Human Resources Managers', ["lca_case_soc_code"]] = '11-3121' # from ONET
h1b.loc[h1b.lca_case_soc_name == 'Purchasing Managers', ["lca_case_soc_code"]] = '11-3061'
h1b.loc[h1b.lca_case_soc_name == 'Emergency Management Directors', ["lca_case_soc_code"]] = '11-9161'
h1b.loc[h1b.lca_case_soc_name == 'Transportation, Storage, and Distribution Managers', ["lca_case_soc_code"]] = '11-3071' # from ONET
h1b.loc[h1b.lca_case_soc_name == 'Education Administrators, Postsecondary', ["lca_case_soc_code"]] = '11-9033' # from ONET
h1b.loc[h1b.lca_case_soc_name == 'Lodging Managers', ["lca_case_soc_code"]] = '11-9081'
h1b.loc[h1b.lca_case_soc_name == 'Farmers, Ranchers, and Other Agricultural Managers', ["lca_case_soc_code"]] = '11-9013'

In [177]:
# so now I'll replace the codes with all the correct ones XX-XXXX
res = [re.findall(r"^\d{2}-\d{4}", soc) for soc in h1b["lca_case_soc_code"].fillna("---").values]
h1b.loc[:, ["lca_case_soc_code"]] = res

In [178]:
h1b.loc[:, ["lca_case_soc_code"]] = [a[0][0] if len(a[0]) > 0 else "" for a in h1b.loc[:, ["lca_case_soc_code"]].values]

In [179]:
# now match on the names, since people seem to be better at filling those in
# - keep the most common code in groups with the same name
soc_names = h1b.groupby("lca_case_soc_name").size().sort_values(ascending=False).index
for name in soc_names:
    soc_codes = h1b.loc[h1b.lca_case_soc_name == name, ["lca_case_soc_code"]]
    soc_rank = soc_codes.groupby("lca_case_soc_code").size().sort_values(ascending=False)
    if soc_rank.shape[0] > 1:
        h1b.loc[h1b.lca_case_soc_name == name, ["lca_case_soc_code"]] = soc_rank.index[0]

In [180]:
# as with naics code, the first two numbers are the most informative - SOC code family
res = [re.findall(r"^\d{2}", soc) for soc in h1b["lca_case_soc_code"].fillna("---").values]
h1b["soc_fam"] = [a[0] if len(a) > 0 else "" for a in res]

In [147]:
h1b.groupby("soc_fam").size().sort_values(ascending=False)

soc_fam
15    300010
13     37718
17     31197
19     16662
29     16210
11     16167
25     11686
27      7481
41      2192
23      1359
21       938
43       402
         251
45       130
39       107
35       106
51        79
31        77
53        65
49        36
47        19
37         9
33         6
24         1
14         1
60         1
dtype: int64

In [148]:
# What kind of info is in the job title?
# There definitely seems to be some redundancy here... what to do with that
h1b.groupby("lca_case_job_title").size().sort_values(ascending=False)

lca_case_job_title
PROGRAMMER ANALYST                                    38545
SOFTWARE ENGINEER                                     17203
COMPUTER PROGRAMMER                                   13621
SYSTEMS ANALYST                                        9129
SOFTWARE DEVELOPER                                     6417
BUSINESS ANALYST                                       6313
COMPUTER SYSTEMS ANALYST                               6145
TECHNOLOGY LEAD - US                                   5055
TECHNOLOGY ANALYST - US                                4911
SENIOR CONSULTANT                                      4539
CONSULTANT                                             4072
SENIOR SOFTWARE ENGINEER                               3973
ASSISTANT PROFESSOR                                    3319
PROJECT MANAGER                                        2905
WEB DEVELOPER                                          2792
SYSTEMS ENGINEER - US                                  2463
DATABASE ADMINISTRATO

In [181]:
# okay, first the silly US suffixes
h1b.loc[:, ["lca_case_job_title"]] = [a[:-4] if a[-4:] == "- US" else a for a in h1b.loc[:, ["lca_case_job_title"]].values.flatten()]

In [159]:
# I think the main thing which is useful here, is terms like "MANAGER", "LEAD", "2" , "II", "III", "3"
# This adds more info than what is in the soc code
h1b.groupby("lca_case_job_title").size().sort_values(ascending=False).iloc[250:300]

lca_case_job_title
SOFTWARE QA ENGINEER                                  188
ASSOCIATE CONSULTANT                                  188
SR. SYSTEMS ANALYST                                   187
MARKETING ANALYST                                     186
COMPUTER PROGRAMMER/CONFIGURER 3                      184
TECHNICAL MANAGER                                     183
ASSOCIATE ATTORNEY                                    183
SOFTWARE QUALITY ASSURANCE ENGINEERS AND TESTERS      181
SCIENTIST                                             181
APPLICATION PROGRAMMER                                181
ENGINEER II                                           179
INTERNAL MEDICINE PHYSICIAN                           179
LAW CLERK                                             179
SENIOR DATABASE ADMINISTRATOR                         179
PUBLIC RELATIONS SPECIALIST                           178
OPERATIONS MANAGER                                    177
TECHNICAL SUPPORT ENGINEER                           

In [160]:
set([a for a in h1b.loc[:, ["lca_case_job_title"]].values.flatten() if a[:9] == "ASSOCIATE"])

{'ASSOCIATE RADIATION PHYSICIST',
 'ASSOCIATE MODEL RISK & DEVELOPMENT',
 'ASSOCIATE MANAGER, ENTERPRISE LIMS',
 'ASSOCIATE DIRECTOR, ANALYST',
 'ASSOCIATE SCIENTIFIC DIRECTOR',
 'ASSOCIATE CONSTRUCTION ENGINEER/SPECIALIST',
 'ASSOCIATE HEAD COACH/RECRUITING COORDINATOR',
 'ASSOCIATE DIRECTOR, SENIOR DEVELOPER',
 'ASSOCIATE DEAN AND DIRECTOR',
 'ASSOCIATE PROFESSOR OF CLINICAL PEDIATRICS',
 'ASSOCIATE PARTNER: ERP PRODUCT',
 'ASSOCIATE/SENIOR COST ESTIMATOR',
 'ASSOCIATE PROJECT MANAGER, ENGINEERING',
 'ASSOCIATE AGENT',
 'ASSOCIATES',
 'ASSOCIATE SOLUTIONS DEVELOPER',
 'ASSOCIATE PROFESSOR/FULL PROFESSOR',
 'ASSOCIATE INTERACTIVE MEDIA DESIGNER',
 'ASSOCIATE MANAGER, DRUG SAFETY & PUBLIC HEALTH',
 'ASSOCIATE RADIO BROADCAST NEGOTIATOR',
 'ASSOCIATE, ANALYTICS',
 'ASSOCIATE, SOLUTIONS ARCHITECT',
 'ASSOCIATE SCIENTIST II, ANALYTICAL CHEMISTRY',
 'ASSOCIATE, U.S. PORTFOLIO CONSTRUCTION',
 'ASSOCIATE FELLOW',
 'ASSOCIATE RISK OFFICER I',
 'ASSOCIATE, APPLICATIONS SUPPORT ANALYST',
 'ASSO

In [182]:
seniorities = ["SENIOR", "MANAGER", "LEAD", " 2" , "II", "III", " 3"]
set([a for a in h1b.loc[:, ["lca_case_job_title"]].values.flatten() if any(x in a for x in seniorities)])

{'COMPUTER & INFORMATION SYSTEMS MANAGER (GAME DESIG',
 'INTERNATIONAL BUSINESS MANAGER',
 'PEOPLESOFT SENIOR CONSULTANT',
 'SENIOR FIELD DESIGN ENGINEER - GPS',
 'MANAGER - CREDIT RISK MODELER',
 'ACES ANALYST II',
 'SENIOR CUSTOMER INFORMATION MANAGEMENT ANALYST',
 'SENIOR SQL SERVER/BI DEVELOPER',
 'SENIOR ANALYST IN MERCHANDISING/DEMAND PLANNING',
 'SCIENTIFIC ASSISTANT SENIOR',
 'ENGINEER SOFTWARE APPS - SENIOR',
 'TESTING GROUP LEADER',
 'INFRASTRUCTURE ENGINEER SENIOR SPECIALIST',
 'COMPUTER SYSTEMS ANALYST (SYSTEMS AUTOMATION LEAD)',
 'SENIOR SCIENCE ADVISER',
 'APPLICATION DELIVERY SENIOR PROFESSIONAL',
 'PROGRAMS MANAGER',
 'SERVICE OPERATIONS 2',
 'SENIOR SERVER TECHNOLOGY SPECIALIST',
 'SENIOR DATABASE ADMINISTRATOR AND ENGINEER',
 'IS SENIOR APPLICATION ARCHITECT',
 'SYSTEM ADMINISTRATOR LEVEL 3',
 'SENIOR MECHANICAL ENGINEER - HEAT TRANSFER',
 'EXECUTIVE OFFICE MANAGER',
 'MANAGER, MICROBIOLOGY DEVELOPMENT',
 'LEAD CONSUTLANT',
 'MANAGER, BUSINESS ENGAGEMENT',
 'PROSOURCE

In [183]:
h1b["seniority"] = [1 if any(x in a for x in seniorities) else 0 for a in h1b.loc[:, ["lca_case_job_title"]].values.flatten()]

In [184]:
h1b = h1b.dropna(subset=["lca_case_wage_rate_unit", "pw_unit_1"]) # in case there's still any of these

In [185]:
# Alright now salaries
# everything becomes annual
h1b.loc[h1b.lca_case_wage_rate_unit == "Hour", ["lca_case_wage_rate_from"]] = 24 * 52 * h1b.loc[h1b.lca_case_wage_rate_unit == "Hour", ["lca_case_wage_rate_from"]]
h1b.loc[h1b.lca_case_wage_rate_unit == "Hour", ["lca_case_wage_rate_to"]] = 24 * 52 * h1b.loc[h1b.lca_case_wage_rate_unit == "Hour", ["lca_case_wage_rate_to"]]
h1b.loc[h1b.lca_case_wage_rate_unit == "Week", ["lca_case_wage_rate_from"]] = 52 * h1b.loc[h1b.lca_case_wage_rate_unit == "Week", ["lca_case_wage_rate_from"]]
h1b.loc[h1b.lca_case_wage_rate_unit == "Week", ["lca_case_wage_rate_to"]] = 52 * h1b.loc[h1b.lca_case_wage_rate_unit == "Week", ["lca_case_wage_rate_to"]]
h1b.loc[h1b.lca_case_wage_rate_unit == "Bi-Weekly", ["lca_case_wage_rate_from"]] = 26 * h1b.loc[h1b.lca_case_wage_rate_unit == "Bi-Weekly", ["lca_case_wage_rate_from"]]
h1b.loc[h1b.lca_case_wage_rate_unit == "Bi-Weekly", ["lca_case_wage_rate_to"]] = 26 * h1b.loc[h1b.lca_case_wage_rate_unit == "Bi-Weekly", ["lca_case_wage_rate_to"]]
h1b.loc[h1b.lca_case_wage_rate_unit == "Month", ["lca_case_wage_rate_from"]] = 12 * h1b.loc[h1b.lca_case_wage_rate_unit == "Month", ["lca_case_wage_rate_from"]]
h1b.loc[h1b.lca_case_wage_rate_unit == "Month", ["lca_case_wage_rate_to"]] = 12 * h1b.loc[h1b.lca_case_wage_rate_unit == "Month", ["lca_case_wage_rate_to"]]

h1b.loc[h1b.pw_unit_1 == "Hour", ["pw_1"]] = 24 * 52 * h1b.loc[h1b.pw_unit_1 == "Hour", ["pw_1"]]
h1b.loc[h1b.pw_unit_1 == "Week", ["pw_1"]] = 52 * h1b.loc[h1b.pw_unit_1 == "Week", ["pw_1"]]
h1b.loc[h1b.pw_unit_1 == "Bi-Weekly", ["pw_1"]] = 26 * h1b.loc[h1b.pw_unit_1 == "Bi-Weekly", ["pw_1"]]
h1b.loc[h1b.pw_unit_1 == "Month", ["pw_1"]] = 12 * h1b.loc[h1b.pw_unit_1 == "Month", ["pw_1"]]

h1b.loc[h1b.pw_unit_2 == "Hour", ["pw_2"]] = 24 * 52 * h1b.loc[h1b.pw_unit_2 == "Hour", ["pw_2"]]
h1b.loc[h1b.pw_unit_2 == "Week", ["pw_2"]] = 52 * h1b.loc[h1b.pw_unit_2 == "Week", ["pw_2"]]
h1b.loc[h1b.pw_unit_2 == "Bi-Weekly", ["pw_2"]] = 26 * h1b.loc[h1b.pw_unit_2 == "Bi-Weekly", ["pw_2"]]
h1b.loc[h1b.pw_unit_2 == "Month", ["pw_2"]] = 12 * h1b.loc[h1b.pw_unit_2 == "Month", ["pw_2"]]

In [186]:
h1b = h1b.drop(columns=["lca_case_wage_rate_unit", "pw_unit_1", "pw_unit_2"])

In [195]:
# lastly I think it would be good to figure out what's worth keeping in "wage_rate_to"
# let's look at the distribution of differences in any rows that have it
h1b["frac_wage_rate_to"] = ((h1b["lca_case_wage_rate_to"] - h1b["lca_case_wage_rate_from"]) / h1b["lca_case_wage_rate_from"]).fillna(0)
h1b["frac_wage_rate_to"].describe()

count    442910.000000
mean          0.060171
std           0.177391
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          15.294725
Name: frac_wage_rate_to, dtype: float64

In [196]:
h1b.to_csv("../dat/clean_emp_jobs.csv")